In [1]:
libraries = c("dplyr", "tidyverse", "data.table", "readxl")
for(x in libraries) {library(x,character.only=TRUE,warn.conflicts=FALSE,quietly=TRUE)}

theme_set(theme_bw())

── Attaching packages ───────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   0.3.5
✔ tibble  3.1.8     ✔ stringr 1.4.1
✔ tidyr   1.2.1     ✔ forcats 0.5.2
✔ readr   2.1.3     
── Conflicts ──────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
path <- "../data/flight/all_region/"

options(warn=-1)
list.files(path = path, pattern = "*xlsx") -> file_list
read_xlsx_files <- function(x){
    df <- suppressMessages(read_xlsx(path = paste0(path, x), skip = 5))
    df[-1, c(1,2,27)] -> df_2019
    colnames(df_2019) <- c("destination", "series", "v2019")
    return(df_2019)}
lapply(file_list, read_xlsx_files) -> df_list

for(i in 1:length(df_list)){
    colnames(df_list[[i]]) <- c("destination", "series", substr(file_list[i],1,nchar(file_list[i])-5))
}

options(warn=0)

In [3]:
merge.all <- function(x, y) {merge(x, y, all=TRUE, by=c("destination", "series"))}
output <- Reduce(merge.all, df_list)

output %>% filter(!is.na(destination)) -> output

In [4]:
output %>% filter(destination %in% c("Czechia", "Czech Republic (Czechia)")) -> temp
unique(temp$series) -> series_list
Czech_list <- list()
for(i in 1:length(series_list)){
    temp %>% filter(series==series_list[i]) -> temp; temp[is.na(temp)] <- 0
    temp %>% dplyr::select(-c("destination", "series")) -> temp_col
    as.data.frame(colSums(temp_col))%>% t() %>% as.data.frame() %>%
    mutate(destination=c("Czech Republic"), series=series_list[i]) -> temp_all; rownames(temp_all) <- NULL
    temp_all -> Czech_list[[i]]
}

do.call(rbind, Czech_list) -> Czech_output

In [5]:
output %>% filter(destination %in% c("Türkiye", "Turkey")) -> temp
unique(temp$series) -> series_list
Turkey_list <- list()
for(i in 1:length(series_list)){
    temp %>% filter(series==series_list[i]) -> temp; temp[is.na(temp)] <- 0
    temp %>% dplyr::select(-c("destination", "series")) -> temp_col
    as.data.frame(colSums(temp_col))%>% t() %>% as.data.frame() %>%
    mutate(destination=c("Turkey"), series=series_list[i]) -> temp_all; rownames(temp_all) <- NULL
    temp_all -> Turkey_list[[i]]
}

do.call(rbind, Turkey_list) -> Turkey_output

In [6]:
output %>% filter(!(destination %in% c("END", "Türkiye", "Turkey", "Czechia", "Czech Republic (Czechia)"))) %>%
mutate(destination=case_when(destination==c("Curaçao")~c("Curacao"), 
                             destination==c("Côte d\'Ivoire")~c("Cote d'Ivoire"), 
                             destination==c("Taiwan Province of China")~c("Taiwan, Province of China"), 
                             destination==c("Saint Vincent and the Grenadines")~c("Saint Vincent and The Grenadines"),
                             destination==c("Palestine")~c("Palestine, State of"),
                             TRUE~destination)) -> output

rbind(output, Turkey_output, Czech_output) -> output

In [7]:
## coverting NAs to zero in the flight data
output[is.na(output)] <- 0
output %>% mutate(total_vol = rowSums(output[,3:ncol(output)])) %>% 
filter(total_vol > 0) %>% dplyr::select(-total_vol) -> output

In [8]:
## selecting the biggest value throughout the series
colMax <- function(data) sapply(data, max, na.rm = TRUE)

new_list <- list()
unique(output$destination) -> country_list


for (g in 1:length(country_list)){
    output %>% filter(destination==country_list[g]) -> temp
    as.data.frame(t(colMax(temp)))-> new_list[[g]]
}

            
do.call("rbind", new_list) -> flight_matrix
write.csv(flight_matrix, "../data/flight/flight_matrix.csv")     

In [9]:
flight_matrix %>% filter(destination==("Palestine, State of")) -> temp
temp

destination,series,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,⋯,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Vanuatu,"Venezuela, Bolivarian Republic of",Viet Nam,Yemen,Zambia,Zimbabwe
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
"Palestine, State of",THSN,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [10]:
setdiff(flight_matrix$destination, colnames(flight_matrix[,3:ncol(flight_matrix)]))
setdiff(colnames(flight_matrix[,3:ncol(flight_matrix)]), flight_matrix$destination)

[1] "Cabo Verde"                   "Eswatini"                    
[3] "Reunion"                      "Sint Maarten (Dutch part)"   
[5] "Tanzania, United Republic of"

[1] "Afghanistan"                          
 [2] "American Samoa"                       
 [3] "Botswana"                             
 [4] "British Virgin Islands"               
 [5] "Burundi"                              
 [6] "Cape Verde"                           
 [7] "Chad"                                 
 [8] "Congo, Democratic Republic of the"    
 [9] "Congo"                                
[10] "Djibouti"                             
[11] "Equatorial Guinea"                    
[12] "Eritrea"                              
[13] "Gabon"                                
[14] "Ghana"                                
[15] "Guadeloupe"                           
[16] "Guinea-Bissau"                        
[17] "Guinea"                               
[18] "Iraq"                                 
[19] "Korea, Democratic Peoples Republic of"
[20] "Liberia"                              
[21] "Libyan Arab Jamahiriya"               
[22] "Malawi"                               
[23] "Marshall Islands"                     
[24] "Mauritania"                           
[25] "Micronesia (Federated States Of)"     
[26] "Nauru"                                
[27] "Nigeria"                              
[28] "Niue"                                 
[29] "Pakistan"                             
[30] "Sao Tome and Principe"                
[31] "Senegal"                              
[32] "Somalia"                              
[33] "Sudan"                                
[34] "Suriname"                             
[35] "Swaziland"                            
[36] "Turkmenistan"                         
[37] "Uganda"                               
[38] "Venezuela, Bolivarian Republic of"    
[39] "Yemen"

In [11]:
#### setting the threshold for the eligible country selection
thres <- 1 ## proportion of NAs (1 for all allowing all NAs)

## destination (V_ij)
flight_matrix %>% rowwise() %>%
mutate(prop=sum(c_across()==c("0"))/(ncol(flight_matrix)-2)) %>% filter(prop <= thres) %>% 
dplyr::select(-prop) -> flight_matrix_desti_thres
write.csv(flight_matrix_desti_thres, "../data/flight/flight_matrix_desti_thres.csv")

## departure (V_ji)
t(flight_matrix_desti_thres) %>% as.data.frame()-> temp
temp[1,] -> tempp; rownames(tempp) <- NULL
colnames(temp) <- tempp[1,]
temp[-(1:2),] -> temp
temp %>% mutate(departure=rownames(temp)) -> temp; rownames(temp) <- NULL 
temp %>% rowwise() %>% mutate(prop=sum(c_across()==c("0"))/(ncol(temp)-1)) %>% filter(prop <= thres) %>% 
dplyr::select(-prop) -> flight_matrix_depart_thres
write.csv(flight_matrix_depart_thres, "../data/flight/flight_matrix_depart_thres.csv")

In [23]:
flight_matrix_depart_thres %>% filter(departure==c("United States of America")) %>% dplyr::select(-departure) -> temp
as.numeric(temp) -> temp

sum(temp)

[1] 143769174

In [19]:
flight_matrix_depart_thres

Albania,Algeria,Andorra,Angola,Anguilla,Antigua and Barbuda,Argentina,Armenia,Aruba,Australia,⋯,United States Virgin Islands,United States of America,Uruguay,Uzbekistan,Vanuatu,Viet Nam,Zambia,Zimbabwe,Turkey,departure
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
73,0,0,9,0,93,0,50,0,1673,⋯,0,3170,0,62580,0,0,0,0,37190,Afghanistan
0,0,0,10,0,52,0,162,0,413,⋯,0,18715,0,63,0,0,0,0,86555,Albania
193,0,0,318,0,15,0,29,0,503,⋯,0,13523,0,120,0,0,0,0,331150,Algeria
0,0,0,0,0,0,0,0,0,12,⋯,0,0,0,0,0,0,0,0,2786,American Samoa
84,0,0,0,0,6,0,0,0,324,⋯,0,2218,0,29,0,0,0,0,842,Andorra
0,0,0,0,0,5,0,23,0,268,⋯,0,6620,0,8,0,0,0,5050,3265,Angola
0,0,0,0,0,579,0,0,0,19,⋯,0,1988,0,2,0,0,0,0,399,Anguilla
205,0,0,0,0,0,0,0,0,0,⋯,0,18229,0,0,0,0,0,0,0,Antigua and Barbuda
2110,425,0,455,0,332,0,1418,15515,23825,⋯,125,854442,1744643,319,0,0,0,1305,100820,Argentina


In [59]:
sum(as.numeric(flight_matrix$Germany))
sum(as.numeric(flight_matrix$Germany))

[1] 153811503

[1] 153811503

In [60]:
t(flight_matrix_desti_thres) %>% as.data.frame()-> temp
temp[1,] -> tempp; rownames(tempp) <- NULL
colnames(temp) <- tempp[1,]
temp[-(1:2),] -> temp
temp %>% mutate(departure=rownames(temp)) -> temp; rownames(temp) <- NULL 
temp %>% rowwise() %>% mutate(prop=sum(c_across()==c("0"))/(ncol(temp)-1)) %>% 
filter(departure %in% c("United Kingdom", "Germany")) -> temp

write.csv(temp, "temp.csv")

In [61]:
flight_matrix_desti_thres %>% filter(destination==c("Japan")) -> temp
sum(as.numeric(temp[3:215]))

[1] 31878634

In [62]:
flight_matrix_depart_thres %>% filter(departure==c("Brazil")) -> temp
sum(as.numeric(temp[1:180]))

[1] 11941519

In [63]:
flight_matrix %>% rowwise() %>%
mutate(prop=sum(c_across()==c("0"))/(ncol(flight_matrix)-2)) %>% filter(prop <= thres) %>%
filter(destination==c("Vanuatu"))

destination,series,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antigua and Barbuda,⋯,United States of America,Uruguay,Uzbekistan,Vanuatu,"Venezuela, Bolivarian Republic of",Viet Nam,Yemen,Zambia,Zimbabwe,prop
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Vanuatu,TFR,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0.9766355
